In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

# Wczytaj dane
df = pd.read_csv("final_dataset_1.csv")

# Usuń 'Unnamed: 0' jeśli istnieje
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# Zapamiętaj oryginalną kolejność kolumn
original_columns = df.columns.tolist()

# Zakoduj FTR jako liczby
le = LabelEncoder()
df['FTR_encoded'] = le.fit_transform(df['FTR'])

# Zamień W/D/L na liczby
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    df[col] = df[col].map(form_map)

# Przygotowanie do undersamplingu
X = df.select_dtypes(include=['number']).drop(columns=['FTR_encoded'])
y = df['FTR_encoded']

# Imputacja braków
X_imputed = SimpleImputer(strategy='mean').fit_transform(X)

# UNDERSAMPLING
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_imputed, y)

# Zamiana na DataFrame
X_res_df = pd.DataFrame(X_resampled, columns=X.columns)
y_res_df = pd.Series(le.inverse_transform(y_resampled), name='FTR')

# Losowe metadane do zachowania struktury
meta_cols = ['Date', 'HomeTeam', 'AwayTeam', 'HTFormPtsStr', 'ATFormPtsStr']
meta_sample = df[meta_cols].sample(n=len(X_res_df), replace=True, random_state=42).reset_index(drop=True)

# Połączenie danych
final_df = pd.concat([meta_sample, X_res_df.reset_index(drop=True), y_res_df], axis=1)

# Uzupełnij brakujące kolumny
for col in original_columns:
    if col not in final_df.columns:
        final_df[col] = ""

# Przywrócenie oryginalnej kolejności kolumn
final_df = final_df[original_columns]

# Zaokrąglenie logicznych kolumn (gole, streaki) do int
int_cols = ['FTHG', 'FTAG', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'MW',
            'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
            'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5']
for col in int_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].round().astype(int)

# Zapis do CSV bez indeksu
final_df.to_csv("final_dataset_undersampled_clean.csv", index=False)
print("✅ Zapisano: final_dataset_undersampled_clean.csv")


✅ Zapisano: final_dataset_undersampled_clean.csv


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import numpy as np

# Wczytaj dane
df = pd.read_csv("final_dataset_1.csv")

# Usuń kolumnę indeksu jeśli istnieje
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# Zapisz oryginalny porządek kolumn
original_columns = df.columns.tolist()

# Zakoduj etykietę FTR
le = LabelEncoder()
df['FTR_encoded'] = le.fit_transform(df['FTR'])  # H=0, D=1, A=2

# Zamień W/D/L na liczby (dla streaków)
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    if col in df.columns:
        df[col] = df[col].map(form_map)

# Podział klas zgodnie z zakodowanymi etykietami
df_H  = df[df['FTR'] == 'H'].sample(n=190, random_state=42)
df_D  = df[df['FTR'] == 'D'].sample(n=167, random_state=42)
df_A  = df[df['FTR'] == 'A'].sample(n=198, random_state=42)

# Połączenie podzbiorów
df_balanced = pd.concat([df_H, df_D, df_A]).sample(frac=1, random_state=42).reset_index(drop=True)

# Zaokrąglenie kolumn logicznych do int
int_cols = ['FTHG', 'FTAG', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'MW',
            'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
            'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5']
for col in int_cols:
    if col in df_balanced.columns:
        df_balanced[col] = df_balanced[col].round().astype(int)

# Przywróć oryginalny porządek kolumn
df_balanced = df_balanced[original_columns]

# Zapisz do pliku
df_balanced.to_csv("final_dataset_undersampled_balanced.csv", index=False)
print("✅ Zapisano jako final_dataset_undersampled_balanced.csv")


✅ Zapisano jako final_dataset_undersampled_balanced.csv


In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Wczytaj dane
df = pd.read_csv("final_dataset_1.csv")

# Usuń kolumnę indeksu jeśli istnieje
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

# Zapisz oryginalny porządek kolumn
original_columns = df.columns.tolist()

# Zakoduj etykietę celu (jeśli potrzebna do modelu)
le = LabelEncoder()
df['FTR_encoded'] = le.fit_transform(df['FTR'])  # H=0, D=1, A=2

# Zamiana W/D/L na liczby
form_cols = ['HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']
form_map = {'W': 3, 'D': 1, 'L': 0}
for col in form_cols:
    if col in df.columns:
        df[col] = df[col].map(form_map)

# Sprawdź liczności klas
min_class_size = df['FTR'].value_counts().min()

# Losowe próbkowanie każdej klasy
df_H = df[df['FTR'] == 'H'].sample(n=min_class_size, random_state=42)
df_D = df[df['FTR'] == 'D'].sample(n=min_class_size, random_state=42)
df_A = df[df['FTR'] == 'A'].sample(n=min_class_size, random_state=42)

# Połączenie i przetasowanie
df_balanced = pd.concat([df_H, df_D, df_A]).sample(frac=1, random_state=42).reset_index(drop=True)

# Zaokrąglenie kolumn logicznych (bramki, streaki itd.)
int_cols = ['FTHG', 'FTAG', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'MW',
            'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
            'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5']
for col in int_cols:
    if col in df_balanced.columns:
        df_balanced[col] = df_balanced[col].round().astype(int)

# Przywróć oryginalny porządek kolumn
df_balanced = df_balanced[original_columns]

# Zapisz do pliku CSV bez indeksu
df_balanced.to_csv("final_dataset_undersampled_equal.csv", index=False)
print("✅ Zapisano plik: final_dataset_undersampled_equal.csv")


✅ Zapisano plik: final_dataset_undersampled_equal.csv
